In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           1.17.0-py_0          conda-forge

The following packages will be UPDATED:

    ca-certificates: 2018.8.24-ha4d7672_0 conda-forge --> 2018.10.15-ha4d7672_0 conda-forge
    certifi:         2018.8.24-py36_1001  conda-forge --> 2018.10.15-py36_1000  conda-forge
    conda:           4.5.11-py36_0        conda-forge --> 4.5.11-py36_1000      conda-forge
    openssl:         1.0.2p-h470a237_0    conda-forge --> 1.0.2p-h470a237_1     conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_10

In [2]:
from bs4 import BeautifulSoup

# ASSIGNMENT  - PART 1

Extract table of wikipedia page as several lists.
Append each list to an empty one

In [3]:
myList=list()
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")                                                  
soup = BeautifulSoup(res.text, 'lxml') 
table = soup.find("table",class_="wikitable")

for items in table.find_all("tr")[:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    myList.append(data)
print(myList)

[['Postcode', 'Borough', 'Neighbourhood'], ['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Isl

Trasform list to dataframe:

In [4]:
data_transposed = zip(*myList)
df = pd.DataFrame(myList, columns=['Postcode', 'Borough', 'Neighbourhood'])
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Delete 1st row

In [5]:
df=df.tail(-1)
df.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


Replace string "Not assigned" with NaN values

In [6]:
df3=df.replace("Not assigned", np.nan)
df3.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,NaN,NaN
2,M2A,NaN,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,NaN
10,M8A,NaN,NaN


 Delete rows with a borough that is NaN (Not assigned)

In [8]:
df=df3.dropna(subset=['Borough'])
df.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,NaN
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If a cell has a borough but a NaN (e.g. Not assigned) neighborhood, then the neighborhood will be the same as the borough:

In [11]:
df = df.reset_index(drop=True)

In [12]:
df['Neighbourhood'].mask(df['Neighbourhood'].isnull(), df['Borough'], inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area.
These two rows will be combined into one row with the neighborhoods separated with a comma:

In [13]:
df.set_index(['Postcode', 'Borough'], inplace=True)

In [14]:
df.head(10)

Neighbourhood
Postcode Borough                           
M3A      North York               Parkwoods
M4A      North York        Victoria Village
M5A      Downtown Toronto      Harbourfront
         Downtown Toronto       Regent Park
M6A      North York        Lawrence Heights
         North York          Lawrence Manor
M7A      Queen's Park          Queen's Park
M9A      Etobicoke         Islington Avenue
M1B      Scarborough                  Rouge
         Scarborough                Malvern

In [15]:
df2=df.groupby(['Postcode', 'Borough'], as_index=False)['Neighbourhood'].apply(','.join)
df2.head(10)

Postcode  Borough    
M1B       Scarborough                                    Rouge,Malvern
M1C       Scarborough             Highland Creek,Rouge Hill,Port Union
M1E       Scarborough                  Guildwood,Morningside,West Hill
M1G       Scarborough                                           Woburn
M1H       Scarborough                                        Cedarbrae
M1J       Scarborough                              Scarborough Village
M1K       Scarborough        East Birchmount Park,Ionview,Kennedy Park
M1L       Scarborough                    Clairlea,Golden Mile,Oakridge
M1M       Scarborough    Cliffcrest,Cliffside,Scarborough Village West
M1N       Scarborough                       Birch Cliff,Cliffside West
dtype: object

In [16]:
type(df2)

pandas.core.series.Series

In [17]:
df2 = df2.to_frame().reset_index()
df2 = df2.rename(columns= {0: 'Neighbourhood'})
df2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [18]:
type(df2)

pandas.core.frame.DataFrame

In [19]:
df2.shape

(103, 3)

# ASSIGNMENT - PART 2

READ CSV file that has the geographical coordinates of each postal code 

In [20]:
csv=pd.read_csv("https://cocl.us/Geospatial_data")
csv

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Concatenate the dataframes to one "res" and drop column "Postal Code"

In [21]:
res = pd.concat([df2, csv], axis=1)
res

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


Delete duplicate "Postal Code" column

In [22]:
res.drop('Postal Code', axis=1, inplace=True)
res

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [23]:
res.shape

(103, 5)

# ASSIGNMENT - PART 3

Explore and cluster the neighborhoods in Toronto

In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(res['Borough'].unique()),
        res.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Names of 11 Boroughs, are:

In [26]:
g = res['Borough'].unique()
g

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

Use geopy library to get the latitude and longitude values of Toronto.

In [27]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(res['Latitude'], res['Longitude'], res['Borough'], res['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods that contains word "York":
North York, East York and York
So let's slice the original dataframe and create a new dataframe.

In [113]:
york=res[res['Borough'].str.contains("York")].reset_index(drop=True)
york.drop('Postcode', axis=1, inplace=True)
york.head(10)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Hillcrest Village,43.803762,-79.363452
1,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,North York,Bayview Village,43.786947,-79.385975
3,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,North York,"Newtonbrook,Willowdale",43.789053,-79.408493
5,North York,Willowdale South,43.770120,-79.408493
6,North York,York Mills West,43.752758,-79.400049
7,North York,Willowdale West,43.782736,-79.442259
8,North York,Parkwoods,43.753259,-79.329656
9,North York,Don Mills North,43.745906,-79.352188


In [114]:
address = 'York, CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York are 43.6896191, -79.479188.


In [115]:
# create map of York using latitude and longitude values
map_York = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york['Latitude'], york['Longitude'], york['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_York)  
    
map_York

Define Foursquare Credentials and Version

In [116]:
CLIENT_ID = '10GTYZCZW3GIIJJ4QUGBRRMDM4YF3X5YCXHTMQ2MU3OBWTGI' # your Foursquare ID
CLIENT_SECRET = 'MCIKTZ0SUMMGJLX10XVEIR5DSMVCJCDI1OWT0GQJUJVPZ13B' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 10GTYZCZW3GIIJJ4QUGBRRMDM4YF3X5YCXHTMQ2MU3OBWTGI
CLIENT_SECRET:MCIKTZ0SUMMGJLX10XVEIR5DSMVCJCDI1OWT0GQJUJVPZ13B


Let's explore the first neighborhood in our dataframe.

In [117]:
york.loc[0, 'Neighbourhood']

'Hillcrest Village'

Get the neighborhood's latitude and longitude values.

In [118]:
neighborhood_latitude = york.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = york.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


Now, let's get the top 50 venues that are in Hillcrest Village within a radius of 900 meters.

In [119]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 900 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=10GTYZCZW3GIIJJ4QUGBRRMDM4YF3X5YCXHTMQ2MU3OBWTGI&client_secret=MCIKTZ0SUMMGJLX10XVEIR5DSMVCJCDI1OWT0GQJUJVPZ13B&v=20180604&ll=43.8037622,-79.3634517&radius=900&limit=50'

In [120]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd8644c4434b9406d6ed330'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 43.81186220810001,
    'lng': -79.35224937368095},
   'sw': {'lat': 43.79566219189999, 'lng': -79.37465402631905}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd9842be914a593adbd56fa',
       'name': 'Tastee',
       'location': {'address': '3913 Don Mills Rd.',
        'crossStreet': 'at Cliffwood Rd.',
        'lat': 43.80772211146167,
        'lng': -79.35679781099806,
        'labeledLatLngs': [{'label': 'display',
      

It seems, that 19 different venues are listed

In [121]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [122]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Tastee,Bakery,43.807722,-79.356798
1,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,Korean Restaurant,43.798391,-79.369187
2,Galati,Grocery Store,43.797831,-79.369410
3,Cummer Park,Park,43.799564,-79.371175
4,Tim Hortons,Coffee Shop,43.798945,-79.369644
5,TD Canada Trust,Bank,43.798466,-79.368832
6,Pizza Pizza,Pizza Place,43.799079,-79.369449
7,Subway,Sandwich Place,43.799047,-79.369438
8,Shoppers Drug Mart,Pharmacy,43.798798,-79.369411
9,Purple Sageway,Housing Development,43.809332,-79.364604


In [123]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

19 venues were returned by Foursquare.


Now, let's explore Neighborhoods in York:

Let's create a function to repeat the same process to all the neighborhoods in York

In [124]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [125]:
york_venues = getNearbyVenues(names=york['Neighbourhood'],
                                   latitudes=york['Latitude'],
                                   longitudes=york['Longitude']
                                  )

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Maple Leaf Park,North Park,Upwood Park
Del Ray,Keelsdale,Mount Dennis,Silverthorn
The Junction North,Runnymede
Humber Summit
Emery,Humberlea
Weston


In [126]:
print(york_venues.shape)
york_venues.head()

(315, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


Let's check how many venues were returned for each neighborhood

In [127]:
york_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto,Downsview East",3,3,3,3,3,3
Caledonia-Fairbanks,6,6,6,6,6,6
"Del Ray,Keelsdale,Mount Dennis,Silverthorn",7,7,7,7,7,7
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [128]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 124 uniques categories.


Now, let's analyze Each Neighborhood

In [129]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighbourhood'] = york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
york_onehot.shape

(315, 125)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [131]:
york_grouped = york_onehot.groupby('Neighbourhood').mean().reset_index()
york_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.055556,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.055556,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.111111,0.000000,0.0,0.00,0.000000,0.000000,0.055556,0.00,0.00000,0.000000,0.055556,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.055556,0.000000,0.00,0.055556,0.000000,0.055556,0.000000,0.00000,0.000000,0.000000,0.055556,0.00,0.055556,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.055556,0.000000,0.000000,0.00,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.250000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.250000,0.00,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0416

In [132]:
york_grouped.shape

(33, 125)

Let's print each neighborhood along with the top 5 most common venues

In [133]:
num_top_venues = 5

for hood in york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
            venue  freq
0     Coffee Shop  0.11
1   Deli / Bodega  0.06
2  Sandwich Place  0.06
3        Pharmacy  0.06
4     Supermarket  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3              Pharmacy  0.04
4                  Café  0.04


----CFB Toronto,Downsview East----
         venue  freq
0      Airport  0.33
1         Park  0.33
2     Bus Stop  0.33
3         Pool  0.00
4  Pizza Place  0.00


----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1  Fast Food Restaurant  0.17
2         Women's Store  0.17
3              Pharmacy  0.17
4                Market  0.17


----

Let's put that into a *pandas* dataframe. 
First, let's write a function to sort the venues in descending order.

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = york_grouped['Neighbourhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pharmacy,Supermarket,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Pet Store,Pizza Place,Restaurant
1,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dessert Shop,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Comfort Food Restaurant,Pub,Butcher,Pizza Place,Pharmacy,Juice Bar
3,"CFB Toronto,Downsview East",Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
4,Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Department Store,Diner,Dim Sum Restaurant,Dessert Shop,Curling Ice
5,"Del Ray,Keelsdale,Mount Dennis,Silverthorn",Fast Food Restaurant,Coffee Shop,Fried Chicken Joint,Sandwich Place,Turkish Restaurant,Restaurant,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop
6,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Yoga Studio,Dim Sum Restaurant,Dog Run,Discount Store,Diner
7,Downsview Central,Food Truck,Home Service,Baseball Field,Yoga Studio,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
8,Downsview Northwest,Liquor Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Department Store,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
9,Downsview West,Grocery Store,Park,Shopping Mall,Bank,Yoga Studio,Department Store,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


##### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [136]:
york_grouped.head(2)

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.055556,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.055556,0.0,0.055556,0.0,0.0,0.0,0.0,0.055556,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [137]:
york_grouped.shape

(33, 125)

In [178]:
# set number of clusters
kclusters = 6

york_grouped_clustering = york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 2, 2, 0, 0, 4, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [168]:
york.shape

(33, 5)

In [179]:
york_merged = york

# add clustering labels
york_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

york_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Toy / Game Store,Women's Store,Video Game Store,Salon / Barbershop,Kids Store,Wings Joint
2,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dessert Shop,Dog Run,Discount Store,Diner,Dim Sum Restaurant
3,North York,"Silver Hills,York Mills",43.757490,-79.374714,0,Cafeteria,Yoga Studio,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop
4,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,North York,Willowdale South,43.770120,-79.408493,2,Restaurant,Ramen Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Café,Plaza,Bubble Tea Shop,Indonesian Restaurant,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [180]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighbourhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

#### Cluster 1

In [181]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Fairview,Henry Farm,Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Toy / Game Store,Women's Store,Video Game Store,Salon / Barbershop,Kids Store,Wings Joint
2,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Yoga Studio,Dessert Shop,Dog Run,Discount Store,Diner,Dim Sum Restaurant
3,"Silver Hills,York Mills",Cafeteria,Yoga Studio,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop
6,York Mills West,Park,Bank,Yoga Studio,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store
7,Willowdale West,Pharmacy,Coffee Shop,Butcher,Grocery Store,Pizza Place,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dog Run
9,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Yoga Studio,Dim Sum Restaurant,Dog Run,Discount Store,Diner
11,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pharmacy,Supermarket,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Pet Store,Pizza Place,Restaurant
13,"CFB Toronto,Downsview East",Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
14,Downsview West,Grocery Store,Park,Shopping Mall,Bank,Yoga Studio,Department Store,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
15,Downsview Central,Food Truck,Home Service,Baseball Field,Yoga Studio,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store


#### Cluster 2

In [182]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Flemingdon Park,Don Mills South",Gym,Asian Restaurant,Coffee Shop,Beer Store,Chinese Restaurant,Sporting Goods Shop,Grocery Store,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant
33,Weston,Park,Convenience Store,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store


#### Cluster 3

In [183]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Newtonbrook,Willowdale",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Willowdale South,Restaurant,Ramen Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Café,Plaza,Bubble Tea Shop,Indonesian Restaurant,Japanese Restaurant
23,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Comfort Food Restaurant,Pub,Butcher,Pizza Place,Pharmacy,Juice Bar
28,"Maple Leaf Park,North Park,Upwood Park",Bakery,Construction & Landscaping,Park,Deli / Bodega,Basketball Court,Dessert Shop,Dog Run,Discount Store,Diner,Dim Sum Restaurant


#### Cluster 4

In [184]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Victoria Village,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Coffee Shop,Fried Chicken Joint,Discount Store,Comfort Food Restaurant,Gastropub,Construction & Landscaping


#### Cluster 5

In [185]:
york_merged.loc[york_merged['Cluster Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Parkwoods,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
12,"Northwood Park,York University",Metro Station,Bar,Massage Studio,Coffee Shop,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Electronics Store,Dessert Shop


#### Cluster 6

In [186]:
york_merged.loc[york_merged['Cluster Labels'] == 5, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Lawrence Heights,Lawrence Manor",Clothing Store,Fraternity House,Miscellaneous Shop,Shoe Store,Coffee Shop,Boutique,Event Space,Accessories Store,Furniture / Home Store,Arts & Crafts Store
